# 上下文管理器  
在 Python 开发中，我们经常会使用到 with 语法块，例如在读写文件时，保证文件描述符的正确关闭，避免资源泄露问题。  
## with语法块  
我们在操作一个文件时，代码可以这么写：  

```python
# 打开文件
f = open('file.txt')
for line in f:
    # 读取文件内容 执行其他操作
    # do_something...
# 关闭文件
f.close()
```

这个例子非常简单，就是打开一个文件，然后读取文件中的内容，最后关闭文件释放资源。  
但是，代码这么写会有一个问题：在打开文件后，如果要对读取到的内容进行其他操作，如果操作期间发生了异常，这就会导致文件句柄无法被释放，进而导致资源的泄露。  
也很简单，我们使用 try ... finally 来优化代码：  
```python
# 打开文件
f = open('file.txt')
try:
    for line in f:
        # 读取文件内容 执行其他操作
        # do_something...
finally:
    # 保证关闭文件
    f.close()
```  
这么写的好处是，在读取文件内容和操作期间，无论是否发生异常，都可以保证最后能释放文件资源。  
但这么优化，代码结构会变得很繁琐，每次都要给代码逻辑增加 try ... finally 才可以，可读性变得很差。  

针对这种情况,我们就可以使用 with 语法块来解决这个问题:
```python
with open('file.txt') as f:
    for line in f:
        # do_something...
```
## 上下文管理器  
首先，我们来看一下 with 的语法格式：  
```python
with context_expression [as target(s)]:
    with-body
```  
要想使用 with 语法块，with 后面的的对象需要实现「上下文管理器协议」。  
**什么是「上下文管理器协议」？**  
一个类在 Python 中，只要实现以下方法，就实现了「上下文管理器协议」：  
- __enter__：在进入 with 语法块之前调用，返回值会赋值给 with 的 target  
- __exit__：在退出 with 语法块时调用，一般用作异常处理   
```python
class TestContext:

    def __enter__(self):
        print('__enter__')
        return 1

    def __exit__(self, exc_type, exc_value, exc_tb):
        print('exc_type: %s' % exc_type)
        print('exc_value: %s' % exc_value)
        print('exc_tb: %s' % exc_tb)

with TestContext() as t:
    print('t: %s' % t)
    
# Output:
# __enter__
# t: 1
# exc_type: None
# exc_value: None
# exc_tb: None
```  
在这个例子中，我们定义了 TestContext 类，它分别实现了 __enter__ 和 __exit__ 方法。  
这样依赖，我们就可以把 TestContext 当做一个「上下文管理器」来使用，也就是通过 with TestContext() as t 方式来执行。  